In [1]:
from pprint import pprint
import json

In [3]:
import dendrotweaks as dd

In [4]:
class Segment():

    def __init__(self, idx, position):
        self.idx = idx
        self.position = position
        self.gbar_na = 0.05
        self.vhalf_na = -35

    def update_parameter(self, param_name, distribution_function):
        # In case of a section this would be done for nodes.
        if not hasattr(self, param_name):
            raise ValueError(f"Segment does not have parameter {param_name}")
        value = distribution_function(self.position)
        setattr(self, param_name, value)

class Section():

    def __init__(self, idx, name, position):
        self.idx = idx
        self.name = name
        self.position = position
        self.cm = 10
        self.Ra = 1000
        self.segments = [Segment(0, position)]

    def update_parameter(self, param_name, distribution_function):
        # In case of a section this would be done for nodes.
        if hasattr(self, param_name):
            value = distribution_function(self.position)
            setattr(self, param_name, value)
        else:
            for segment in self.segments:
                segment.update_parameter(param_name, distribution_function)

In [5]:
nodes = [Section(1, 'A', 10), Section(2, 'B', 20), Section(3, 'C', 30)]

In [ ]:
apical = dd.SectionGroup('apical', nodes)
apical

  Elapsed time: 0.0 seconds
  Elapsed time: 0.0 seconds


Group(apical) with 3 nodes
Parameters: ['cm', 'Ra']

In [8]:
for node in apical.nodes:
    print(node.idx, node.cm, node.Ra)

1 1 100
2 1 100
3 1 100


In [7]:
na = Mechanism('na', {'gbar':0.01, 'vhalf': -35})
na

Mechanism(na) with parameters: {'gbar_na': 0.01, 'vhalf_na': -35}

In [8]:
apical.add_mechanism(na)

In [9]:
apical.parameters

{'cm': constant({'value': 1}),
 'Ra': constant({'value': 100}),
 'gbar_na': constant({'value': 0.01}),
 'vhalf_na': constant({'value': -35})}

In [10]:
func = ParametrizedFunction('constant', value=0.06)
apical.replace_distribution_function('gbar_na', func)

In [ ]:
func = DistributionFunction('linear', intercept=0.06)
apical.replace_distribution_function('gbar_na', func)

In [12]:
apical.update_distribution_function_parameters('gbar_na', slope=1, intercept=0.07)

In [13]:
for sec in apical.nodes:
    print(sec.idx, sec.cm, sec.Ra)
    for seg in sec.segments:
        print('  ', seg.idx, seg.gbar_na, seg.vhalf_na)

1 1 100
   0 10.07 -35
2 1 100
   0 20.07 -35
3 1 100
   0 30.07 -35


In [14]:
pprint(apical.to_dict(), sort_dicts=False)

{'group': {'name': 'apical',
           'nodes': [1, 2, 3],
           'parameters': [{'name': 'cm',
                           'distribution_function': {'function': 'constant',
                                                     'parameters': {'value': 1}}},
                          {'name': 'Ra',
                           'distribution_function': {'function': 'constant',
                                                     'parameters': {'value': 100}}},
                          {'name': 'gbar_na',
                           'distribution_function': {'function': 'linear',
                                                     'parameters': {'slope': 1,
                                                                    'intercept': 0.07}}},
                          {'name': 'vhalf_na',
                           'distribution_function': {'function': 'constant',
                                                     'parameters': {'value': -35}}}]}}


In [15]:
print(json.dumps(apical.to_dict(), indent=4))

{
    "group": {
        "name": "apical",
        "nodes": [
            1,
            2,
            3
        ],
        "parameters": [
            {
                "name": "cm",
                "distribution_function": {
                    "function": "constant",
                    "parameters": {
                        "value": 1
                    }
                }
            },
            {
                "name": "Ra",
                "distribution_function": {
                    "function": "constant",
                    "parameters": {
                        "value": 100
                    }
                }
            },
            {
                "name": "gbar_na",
                "distribution_function": {
                    "function": "linear",
                    "parameters": {
                        "slope": 1,
                        "intercept": 0.07
                    }
                }
            },
            {
                "name": "v